# Organism

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, ELU, LeakyReLU, Flatten, Dense, AveragePooling2D

import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

In [24]:
options = {
    'a_filter_size': [(3,3), (5,5), (7,7), (13,13)],
    'a_include_BN': [True, False],
    'a_output_channels': [32, 64, 128, 256],
    'activation_type': [ReLU, ELU, LeakyReLU],
    'b_filter_size': [(3,3), (5,5), (7,7), (13,13)],
    'b_include_BN': [True, False],
    'b_output_channels': [32, 64, 128, 256],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D]
    }

In [25]:
class Organism:
    def __init__(self, chromosome={}, phase=0):
      self.phase = phase
      self.chromosome = chromosome
      self.build_model()
    def build_model(self):
      if self.phase == 0:
        keras.backend.clear_session()
        
        inputs = Input(shape=(32,32,3))
        x = Conv2D(filters=self.chromosome['a_output_channels'],
                    kernel_size=self.chromosome['a_filter_size'],
                    use_bias=self.chromosome['a_include_BN'])(inputs)
        if self.chromosome['a_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)
        if self.chromosome['include_pool']:
            x = self.chromosome['pool_type']()(x)

        x = Conv2D(filters=self.chromosome['b_output_channels'],
                    kernel_size=self.chromosome['b_filter_size'],
                    use_bias=self.chromosome['b_include_BN'])(x)
        if self.chromosome['b_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)

        x = Flatten()(x)
        x = Dense(10, activation='softmax')(x)

        self.model = tf.keras.Model(inputs=[inputs], outputs=[x])
        self.model.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])
      else:
        print('Phase under construction')
    
    def fitnessFunction(self, train_ds, test_ds):
      self.model.fit(train_ds,
                      epochs=5,
                      verbose=1)
      _, self.fitness = self.model.evaluate(test_ds, verbose=0)
    
    def crossover(self, partner):
      child = Organism(chromosome={}, phase=0)
      endpoint = np.random.randint(low=0, high=len(self.chromosome))
      print(endpoint)
      for idx, key in enumerate(self.chromosome):
        if idx <= endpoint:
          child.chromosome[key] = self.chromosome[key]
        else:
          child.chromosome[key] = partner.chromosome[key]
      return child
    
    def mutation(self, mutationRate):
      for idx, key in enumerate(self.chromosome):
        if np.random.rand() <= mutationRate:
          self.chromosome[key] = options[key][np.random.randint(len(options[key]))]

In [26]:
org = Organism(chromosome=random_hyper(), phase=0)

In [27]:
org.model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 64)        256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          200768    
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)         

In [28]:
org.fitnessFunction(train_ds, test_ds)

Epoch 1/5
196/196 [==============================] - 4s 18ms/step - loss: 1.4093 - accuracy: 0.5067
Epoch 2/5
196/196 [==============================] - 4s 18ms/step - loss: 1.0597 - accuracy: 0.6323
Epoch 3/5
196/196 [==============================] - 4s 18ms/step - loss: 0.9466 - accuracy: 0.6727
Epoch 4/5
196/196 [==============================] - 4s 18ms/step - loss: 0.8708 - accuracy: 0.7000
Epoch 5/5
196/196 [==============================] - 3s 18ms/step - loss: 0.8103 - accuracy: 0.7222


# Generation

In [29]:
# Load the training and testing set of CIFAR10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32')
X_train = X_train/255.

X_test = X_test.astype('float32')
X_test = X_test/255.

y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

In [30]:
# Create TensorFlow dataset
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [31]:
def random_hyper():
  return {
    'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
    'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
    'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
    'activation_type': options['activation_type'][np.random.randint(len(options['activation_type']))],
    'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
    'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
    'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
    'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
    'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))]
    }

def softmax(x):
  """Compute softmax values for each sets of scores in x."""
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum()

In [32]:
class Generation:
	def __init__(self, mutationRate, population_size):
		self.population_size = population_size
		self.population = []
		self.generation_number = 0
		self.finished = False
		self.mutationRate = mutationRate

		# creating the first population GENERATION 1
		# can be thought of as the setup function
		for idx in range(self.population_size):
			self.population.append(Organism(random_hyper(), phase=0))
		# calculates the fitness of all the organisms

		self.calcFitness()

	def calcFitness(self):
		# This function is used to calculate the fitness of all
		# the individuals of the population.
		for individuals in self.population:
			individuals.fitnessFunction(train_ds, test_ds)
	
	# genrate the children and place them in the previous generation
	# to make new generation
	def generate(self):
		probability = [ind.fitness for ind in self.population]
		# create children with pairs
		for idx in range (self.population_size):
			parents = np.random.choice(population, size=(2,), replace=False, p=softmax(probability))
			
			A=parents[0]
			B=parents[1]

			child=A.crossover(B)
			child.fitnessFunction()
			child.mutation(self.mutationRate)
			child.fitnessFunction()
			self.population[idx]=child
		self.generation_number+=1

	# evaluates the whole generation's fitness
	def evaluate(self):
		fitness = [ind.fitness for ind in self.population]
		most_fit = max(fitness)
		print('Generation: {}'.format(self.generation_number))
		print('Best fitness: {:0.2f}'.format(most_fit))
		print('Average fitness: {:0.2f}'.format(sum(fitness)/len(fitness)))
		if most_fit > 0.6 :
			self.finished=True
			

In [ ]:
population_size = 10
mutationRate = 0.1
pop = Generation(mutationRate, population_size)

while not(pop.finished):
	pop.generate()
	pop.evaluate()